# 06 - Agregacje i Transformacje

Zaawansowane operacje agregacji i transformacji w Spark.

**Tematy:**
- groupBy + agg (count, sum, avg, min, max, collect_list)
- pivot - tabele krzyżowe
- rollup i cube - agregacje wielopoziomowe
- Transformacje kolumn: when/otherwise, regexp, string functions
- Praca z datami i timestampami
- explode - rozbijanie tablic
- UDF vs wbudowane funkcje

## 1. Setup

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("06_Aggregations_Transformations") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.host", "recommender-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
properties = {
    "user": "recommender",
    "password": "recommender",
    "driver": "org.postgresql.Driver"
}

ratings = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=properties,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
movies = spark.read.jdbc(jdbc_url, "movielens.movies", properties=properties)

# Cache bo będziemy dużo używać
ratings.cache()
movies.cache()
print(f"Ratings: {ratings.count()}, Movies: {movies.count()}")

## 2. groupBy + agg

Podstawowy mechanizm agregacji w Spark. `groupBy` + `agg` pozwala na wiele agregacji naraz.

In [ ]:
# Prosta agregacja
ratings.groupBy("movie_id").count().show(5)

# Skrót - bezpośrednie wywołanie metod
ratings.groupBy("movie_id").avg("rating").show(5)

In [ ]:
# Wiele agregacji naraz z agg()
movie_stats = ratings.groupBy("movie_id").agg(
    count("*").alias("num_ratings"),
    round(avg("rating"), 2).alias("avg_rating"),
    round(stddev("rating"), 2).alias("std_rating"),
    min("rating").alias("min_rating"),
    max("rating").alias("max_rating"),
    round(sum("rating"), 0).alias("total_rating_sum"),
    countDistinct("user_id").alias("unique_users")
)

movie_stats.orderBy(desc("num_ratings")).show(10)

In [ ]:
# collect_list / collect_set - zbierz wartości w tablicę
# Uwaga: collect_list może być kosztowne na dużych danych!
user_movies = ratings.filter(col("user_id") <= 3) \
    .groupBy("user_id").agg(
        collect_list("movie_id").alias("movies_rated"),
        collect_set("rating").alias("unique_ratings")
    )

user_movies.show(truncate=False)

In [ ]:
# Agregacja bez groupBy - globalna
ratings.agg(
    count("*").alias("total_ratings"),
    countDistinct("user_id").alias("unique_users"),
    countDistinct("movie_id").alias("unique_movies"),
    round(avg("rating"), 2).alias("global_avg_rating")
).show()

### Zadanie 1
Policz dla każdego użytkownika: liczbę ocen, średnią, medianę (użyj `percentile_approx`), i rozstęp (max - min).
Pokaż 10 użytkowników z największym rozstępem ocen.

In [ ]:
# Twoje rozwiązanie:


## 3. pivot - tabele krzyżowe

`pivot` zamienia wartości z wierszy na kolumny - świetne do tworzenia macierzy/tabel krzyżowych.

In [ ]:
# Rozkład ocen per rating - ile razy każda ocena została wystawiona
# groupBy().pivot() zmienia wartości kolumny na nazwy kolumn
rating_dist = ratings.groupBy("user_id") \
    .pivot("rating", [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]) \
    .count() \
    .fillna(0)

rating_dist.filter(col("user_id") <= 5).show()

In [ ]:
# Pivot z dodatkową agregacją - średnia ocena per rok i gatunek
# Najpierw przygotujmy dane
ratings_enriched = ratings \
    .withColumn("year", year(col("rating_timestamp")))

# Średnia ocena per rok (ostatnie 5 lat w danych)
yearly_avg = ratings_enriched \
    .filter(col("year").between(2010, 2015)) \
    .groupBy("movie_id") \
    .pivot("year") \
    .agg(round(avg("rating"), 2)) \
    .join(movies.select("movie_id", "title"), "movie_id")

yearly_avg.filter(col("2010").isNotNull() & col("2015").isNotNull()) \
    .show(10, truncate=False)

## 4. rollup i cube - agregacje wielopoziomowe

- `rollup` - agregacje hierarchiczne (od szczegółu do ogółu)
- `cube` - wszystkie możliwe kombinacje agregacji

Wiersze z NULL w kolumnie grupowania = agregacja dla poziomu wyżej.

In [ ]:
# Przygotuj dane z rokiem i zaokrąglonym ratingiem
ratings_prep = ratings \
    .withColumn("year", year(col("rating_timestamp"))) \
    .withColumn("rating_bucket", floor(col("rating")))

# ROLLUP - hierarchiczna agregacja: year -> rating_bucket -> total
ratings_prep.filter(col("year").between(2013, 2015)) \
    .rollup("year", "rating_bucket") \
    .agg(
        count("*").alias("cnt"),
        round(avg("rating"), 2).alias("avg_rating")
    ) \
    .orderBy("year", "rating_bucket") \
    .show(30)

# Wiersze z NULL w rating_bucket = podsumowanie per rok
# Wiersz z NULL w obu = ogólna suma (grand total)

In [ ]:
# CUBE - wszystkie kombinacje
ratings_prep.filter(col("year").between(2014, 2015)) \
    .cube("year", "rating_bucket") \
    .agg(
        count("*").alias("cnt"),
        round(avg("rating"), 2).alias("avg_rating")
    ) \
    .orderBy("year", "rating_bucket") \
    .show(30)

# Różnica: cube daje też agregację per rating_bucket (bez year)
# rollup daje tylko hierarchię: year+bucket -> year -> total

## 5. Transformacje kolumn

### 5.1 when / otherwise - warunkowe wartości

In [ ]:
# Wielostopniowe kategorie
ratings_categorized = ratings.withColumn(
    "sentiment",
    when(col("rating") >= 4.0, "positive")
    .when(col("rating") >= 2.5, "neutral")
    .otherwise("negative")
).withColumn(
    "rating_emoji",
    when(col("rating") == 5.0, "*****")
    .when(col("rating") >= 4.0, "****")
    .when(col("rating") >= 3.0, "***")
    .when(col("rating") >= 2.0, "**")
    .otherwise("*")
)

ratings_categorized.show(10)

# Podsumowanie sentymentów
ratings_categorized.groupBy("sentiment").count().orderBy(desc("count")).show()

### 5.2 Funkcje stringowe

In [ ]:
# Operacje na stringach
movies_transformed = movies.select(
    col("title"),
    lower(col("title")).alias("title_lower"),
    upper(col("title")).alias("title_upper"),
    length(col("title")).alias("title_length"),
    trim(col("title")).alias("title_trimmed"),
    
    # Wyciągnij rok z tytułu
    regexp_extract(col("title"), r"\((\d{4})\)", 1).alias("year"),
    
    # Usuń rok z tytułu
    regexp_replace(col("title"), r"\s*\(\d{4}\)\s*$", "").alias("clean_title"),
    
    # Substring
    substring(col("title"), 1, 20).alias("title_short")
)

movies_transformed.show(10, truncate=False)

In [ ]:
# split i array operations
movies_genres = movies \
    .withColumn("genre_array", split(col("genres"), "\\|")) \
    .withColumn("num_genres", size(split(col("genres"), "\\|"))) \
    .withColumn("first_genre", element_at(split(col("genres"), "\\|"), 1))

movies_genres.select("title", "genres", "genre_array", "num_genres", "first_genre") \
    .show(10, truncate=False)

### 5.3 Praca z datami

In [ ]:
# Operacje na timestamp
ratings_dates = ratings.select(
    col("user_id"),
    col("movie_id"),
    col("rating"),
    col("rating_timestamp"),
    
    year(col("rating_timestamp")).alias("year"),
    month(col("rating_timestamp")).alias("month"),
    dayofmonth(col("rating_timestamp")).alias("day"),
    hour(col("rating_timestamp")).alias("hour"),
    dayofweek(col("rating_timestamp")).alias("day_of_week"),  # 1=Sunday
    weekofyear(col("rating_timestamp")).alias("week"),
    quarter(col("rating_timestamp")).alias("quarter"),
    
    date_format(col("rating_timestamp"), "EEEE").alias("day_name"),
    date_format(col("rating_timestamp"), "yyyy-MM").alias("year_month")
)

ratings_dates.show(10)

In [ ]:
# Analiza: kiedy ludzie oceniają filmy?
# Rozkład ocen po godzinach
ratings.withColumn("hour", hour(col("rating_timestamp"))) \
    .groupBy("hour") \
    .agg(
        count("*").alias("num_ratings"),
        round(avg("rating"), 2).alias("avg_rating")
    ) \
    .orderBy("hour") \
    .show(24)

In [ ]:
# Rozkład po dniach tygodnia
ratings.withColumn("day_name", date_format(col("rating_timestamp"), "EEEE")) \
    .groupBy("day_name") \
    .agg(
        count("*").alias("num_ratings"),
        round(avg("rating"), 2).alias("avg_rating")
    ) \
    .orderBy(desc("num_ratings")) \
    .show()

### Zadanie 2
Stwórz analizę "trendy ocen w czasie":
1. Pogrupuj oceny po miesiącu (year-month)
2. Policz liczbę ocen, średnią i liczbę unikalnych użytkowników
3. Pokaż wyniki posortowane chronologicznie

In [ ]:
# Twoje rozwiązanie:


## 6. explode - rozbijanie tablic na wiersze

`explode` zamienia tablicę na wiele wierszy - jeden wiersz na element tablicy.

In [ ]:
# Rozbij genres na osobne wiersze
movies_exploded = movies \
    .withColumn("genre", explode(split(col("genres"), "\\|")))

movies_exploded.select("movie_id", "title", "genre").show(15)

print(f"Przed explode: {movies.count()} wierszy")
print(f"Po explode: {movies_exploded.count()} wierszy")

In [ ]:
# Ile filmów w każdym gatunku?
genre_counts = movies_exploded.groupBy("genre") \
    .agg(count("*").alias("num_movies")) \
    .orderBy(desc("num_movies"))

genre_counts.show()

In [ ]:
# Średnia ocena per gatunek
genre_ratings = movies_exploded \
    .join(ratings, "movie_id") \
    .groupBy("genre") \
    .agg(
        count("*").alias("num_ratings"),
        round(avg("rating"), 3).alias("avg_rating"),
        countDistinct("movie_id").alias("num_movies")
    ) \
    .orderBy(desc("avg_rating"))

genre_ratings.show()

In [ ]:
# posexplode - explode z indeksem pozycji
movies.withColumn("genre_array", split(col("genres"), "\\|")) \
    .select("title", posexplode(col("genre_array")).alias("position", "genre")) \
    .filter(col("title").like("%Toy Story%")) \
    .show(truncate=False)

## 7. Macierz współwystępowania gatunków

Ciekawe ćwiczenie łączące explode, self-join i pivot.

In [ ]:
# Które gatunki najczęściej współwystępują?
genre_a = movies_exploded.select(
    col("movie_id"), col("genre").alias("genre_a")
)
genre_b = movies_exploded.select(
    col("movie_id"), col("genre").alias("genre_b")
)

# Self-join: te same filmy, różne gatunki
co_occurrence = genre_a.join(genre_b, "movie_id") \
    .filter(col("genre_a") < col("genre_b")) \
    .groupBy("genre_a", "genre_b") \
    .count() \
    .orderBy(desc("count"))

co_occurrence.show(20)

## 8. UDF vs wbudowane funkcje

**Zasada: zawsze preferuj wbudowane funkcje Spark!**

UDF:
- Serializują dane do Pythona → wolniejsze
- Nie podlegają optymalizacji Catalyst
- Używaj tylko gdy nie ma wbudowanej alternatywy

In [ ]:
import time

# Porównanie wydajności: UDF vs wbudowana funkcja

# UDF
@udf(StringType())
def categorize_udf(rating):
    if rating >= 4.0: return "positive"
    elif rating >= 2.5: return "neutral"
    else: return "negative"

# Wbudowane funkcje
def categorize_builtin(rating_col):
    return when(rating_col >= 4.0, "positive") \
        .when(rating_col >= 2.5, "neutral") \
        .otherwise("negative")

# Pomiar UDF
start = time.time()
ratings.withColumn("cat", categorize_udf(col("rating"))).count()
udf_time = time.time() - start

# Pomiar wbudowanej
start = time.time()
ratings.withColumn("cat", categorize_builtin(col("rating"))).count()
builtin_time = time.time() - start

print(f"UDF: {udf_time:.2f}s")
print(f"Wbudowana: {builtin_time:.2f}s")
print(f"UDF wolniejsza {udf_time/builtin_time:.1f}x")

In [ ]:
# Kiedy UDF jest uzasadniony - złożona logika, zewnętrzne biblioteki
@udf(FloatType())
def wilson_score(pos, total):
    """Wilson score confidence interval lower bound.
    Lepszy ranking niż zwykła średnia - uwzględnia liczbę głosów."""
    import math
    if total == 0:
        return 0.0
    z = 1.96  # 95% confidence
    p = pos / total
    denominator = 1 + z * z / total
    centre = p + z * z / (2 * total)
    delta = z * math.sqrt((p * (1 - p) + z * z / (4 * total)) / total)
    return float((centre - delta) / denominator)

# Zastosowanie: ranking filmów z Wilson score
movie_stats = ratings.groupBy("movie_id").agg(
    count("*").alias("total"),
    sum(when(col("rating") >= 4.0, 1).otherwise(0)).alias("positive")
)

movie_ranking = movie_stats \
    .withColumn("wilson", wilson_score(col("positive"), col("total"))) \
    .filter(col("total") >= 100) \
    .join(movies, "movie_id") \
    .orderBy(desc("wilson")) \
    .select("title", "total", "positive", "wilson")

movie_ranking.show(15, truncate=False)

## Zadanie końcowe

Stwórz "Genre Report" - raport per gatunek filmowy:

1. Rozbij gatunki za pomocą explode
2. Dla każdego gatunku policz:
   - Liczbę filmów
   - Liczbę ocen
   - Średnią ocenę
   - Rok najstarszego i najnowszego filmu (wyciągnij z tytułu)
3. Dodaj pivot: średnia ocena per gatunek per dekada (2000s, 2010s)
4. Posortuj po liczbie ocen malejąco

In [ ]:
# Twoje rozwiązanie:


In [ ]:
ratings.unpersist()
movies.unpersist()
spark.stop()